In [3]:
from typing import Final


In [4]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text: Final[str] = f.read()

print(f"length of the dataset in characters: {len(text)}")


length of the dataset in characters: 1115394


In [5]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [6]:
# Get the character set.
chars: Final[list[str]] = sorted(list(set(text)))
vocab_size: Final[int] = len(chars)
print(f"vocab: {''.join(chars)}")
print(f"vocab size: {vocab_size}")


vocab: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [7]:
stoi: Final[dict[str, int]] = {ch: i for i, ch in enumerate(chars)}
itos: Final[dict[int, str]] = {i: ch for ch, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("Hello World!"))
print(decode(encode("Hello World!")))


[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42, 2]
Hello World!


In [8]:
import torch

device: Final[str] = 'cuda' if torch.cuda.is_available() else 'cpu'
data: Final[torch.Tensor] = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data.shape, data.dtype)
print(data[:500])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Split the data into train and validation sets.

n: Final[int] = int(0.9 * len(data))
train_data: Final[torch.Tensor] = data[:n]
val_data: Final[torch.Tensor] = data[n:]
print(f"length of training data: {len(train_data)}")
print(f"length of validation data: {len(val_data)}")


length of training data: 1003854
length of validation data: 111540


In [10]:
block_size: Final[int] = 8
train_data[:block_size + 1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58], device='cuda:0')

In [11]:
x: Final[torch.Tensor] = train_data[:block_size]
y: Final[torch.Tensor] = train_data[1:block_size + 1]
for i in range(block_size):
    context = x[:i + 1]
    target = y[i]
    print(f"input: {context} target: {target}")


input: tensor([18], device='cuda:0') target: 47
input: tensor([18, 47], device='cuda:0') target: 56
input: tensor([18, 47, 56], device='cuda:0') target: 57
input: tensor([18, 47, 56, 57], device='cuda:0') target: 58
input: tensor([18, 47, 56, 57, 58], device='cuda:0') target: 1
input: tensor([18, 47, 56, 57, 58,  1], device='cuda:0') target: 15
input: tensor([18, 47, 56, 57, 58,  1, 15], device='cuda:0') target: 47
input: tensor([18, 47, 56, 57, 58,  1, 15, 47], device='cuda:0') target: 58


In [12]:
from typing import Tuple

torch.manual_seed(1337)
batch_size: Final[int] = 4 # how many independent sequences will we process in parallel?
block_size: Final[int] = 8 # what is the maximum context length for predictions?

def get_batch(split: str) -> Tuple[torch.Tensor, torch.Tensor]:
    data: Final[torch.Tensor] = train_data if split == "train" else val_data
    ix: Final[torch.Tensor] = torch.randint(len(data) - block_size, (batch_size,))
    x: Final[torch.Tensor] = torch.stack([data[i:i + block_size] for i in ix])
    y: Final[torch.Tensor] = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y


In [13]:
xb, yb = get_batch("train")
print("Inputs: ", xb.shape)
print(xb)
print("Targets: ", yb.shape)
print(yb)


Inputs:  torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]], device='cuda:0')
Targets:  torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]], device='cuda:0')


In [14]:
for b_idx in range(batch_size): # for each batch
    for t in range(block_size): # for each time dimension
        context = xb[b_idx, :t + 1]
        target = yb[b_idx, t]
        print(f"input: {context} target: {target}")


input: tensor([24], device='cuda:0') target: 43
input: tensor([24, 43], device='cuda:0') target: 58
input: tensor([24, 43, 58], device='cuda:0') target: 5
input: tensor([24, 43, 58,  5], device='cuda:0') target: 57
input: tensor([24, 43, 58,  5, 57], device='cuda:0') target: 1
input: tensor([24, 43, 58,  5, 57,  1], device='cuda:0') target: 46
input: tensor([24, 43, 58,  5, 57,  1, 46], device='cuda:0') target: 43
input: tensor([24, 43, 58,  5, 57,  1, 46, 43], device='cuda:0') target: 39
input: tensor([44], device='cuda:0') target: 53
input: tensor([44, 53], device='cuda:0') target: 56
input: tensor([44, 53, 56], device='cuda:0') target: 1
input: tensor([44, 53, 56,  1], device='cuda:0') target: 58
input: tensor([44, 53, 56,  1, 58], device='cuda:0') target: 46
input: tensor([44, 53, 56,  1, 58, 46], device='cuda:0') target: 39
input: tensor([44, 53, 56,  1, 58, 46, 39], device='cuda:0') target: 58
input: tensor([44, 53, 56,  1, 58, 46, 39, 58], device='cuda:0') target: 1
input: tenso

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size: int):
        super().__init__()
        self.token_embedding_table: Final[nn.Embedding] = nn.Embedding(
            vocab_size, vocab_size)

    def forward(self, idx: torch.Tensor, target: torch.Tensor | None = None
        )-> Tuple[torch.Tensor, torch.Tensor | None]:

        # idx and target are both (batch_size, block_size) tensors
        # logits: (batch_size, block_size, vocab_size)
        logits: Final[torch.Tensor] = self.token_embedding_table(idx)

        if target is None:
            loss = None
        else:
            # (batch_size, block_size, vocab_size) -> (batch_size * block_size, vocab_size)
            B, T, C = logits.shape
            logits: Final[torch.Tensor] = logits.view(B * T, C)
            # (batch_size, block_size) -> (batch_size * block_size)
            target = target.view(B * T)
            loss = F.cross_entropy(logits, target)
        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        # idx is (B, T) tensor
        for _ in range(max_new_tokens):
            # get the predictions
            logits, _ = self(idx)
            # focus on the last time step
            logits = logits[:, -1, :] # (B, C)
            # Apply softmax to get probabilities
            # Apply softmax to vocab_size dimension
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx

model: Final[BigramLanguageModel] = BigramLanguageModel(vocab_size).to(device)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

# New line character is 0, which is a perfect start sequence for the model.
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
out_idx = model.generate(idx, 100)[0].tolist() # use [0] to get the first batch.
print(decode(out_idx))


torch.Size([32, 65])
tensor(4.8786, device='cuda:0', grad_fn=<NllLossBackward0>)

pYCXxfRkRZd
wc'wfNfT;OLlTEeC K
jxqPToTb?bXAUG:C-SGJO-33SM:C?YI3a
hs:LVXJFhXeNuwqhObxZ.tSVrddXlaSZaNe


In [16]:
# create a PyTorch optimizer
optimizer: torch.optim.AdamW = torch.optim.AdamW(
    model.parameters(), lr=1e-3)


In [17]:
batch_size: int = 32
max_steps: int = 10000

loss = None

for step in range(max_steps):
    # Sample a batch of data
    xb, yb = get_batch("train")
    xb, yb = xb.to(device), yb.to(device)

    # Zero the gradients
    optimizer.zero_grad(set_to_none=True)

    # Forward pass
    logits, loss = model(xb, yb)

    # Backward pass
    loss.backward()

    # Update the weights
    optimizer.step()

assert loss is not None
print(loss.item())


2.394822835922241


In [18]:
# Generate some text
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
out_idx = model.generate(idx, max_new_tokens=400)[0].tolist() # use [0] to get the first batch.
print(decode(out_idx))



Wawice my.

HDERarom oroup
Yowh$Frtof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Tranousel lind me l.
HAule ce hiry:
Supr aisspllw y.
Herindu n Boopetelaves
MP:

Pl, d mothakleo Windo whth eisbyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thiny aleronth,
MadPre?

WISo myr f-NLIERor,
Sb&y, wardsal thes ghesthidin cour ay aney Iry ts I&fr y ce.
J


### The mathematical trick in self-attention

In [19]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channels
x: torch.Tensor = torch.randn(B, T, C).to(device)
x.shape

torch.Size([4, 8, 2])

In [20]:
# Version 1: Using a for loop to calculate the mean of the previous time step.
# We want to use previous time step to predict the next time step.
# The easiest way to do this is to calculate the mean of the previous time step.
# bow (bag of words) is a simple way to do this.
x_bow: torch.Tensor = torch.zeros(
    (B, T, C), device=device
)
for ib in range(B): # for each batch
    for it in range(T): # for each time dimension
        # x[ib, :it + 1]: shape (t, C)
        # x_bow[ib, it]: shape (C,)
        x_prev = x[ib, :it + 1] # (t, C)
        x_bow[ib, it] = torch.mean(x_prev, dim=0)

In [21]:
x[0]


tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]], device='cuda:0')

In [22]:
x_bow[2]


tensor([[-0.6631, -0.2513],
        [ 0.1735, -0.0649],
        [ 0.1685,  0.3348],
        [-0.1621,  0.1765],
        [-0.2312, -0.0436],
        [-0.1015, -0.2855],
        [-0.2593, -0.1630],
        [-0.3015, -0.2293]], device='cuda:0')

In [23]:
torch.manual_seed(42)
# lower triangular matrix can calculate the sum of the previous time step.
a: torch.Tensor = torch.tril(torch.ones(3, 3, device=device))
b: torch.Tensor = torch.randint(0, 10, (3, 2), device=device).float()
c: torch.Tensor = a @ b
print(f"a=\n{a}")
print("------")
print(f"b=\n{b}")
print("------")
print(f"c=\n{c}")


a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]], device='cuda:0')
------
b=
tensor([[3., 9.],
        [8., 0.],
        [3., 7.]], device='cuda:0')
------
c=
tensor([[ 3.,  9.],
        [11.,  9.],
        [14., 16.]], device='cuda:0')


In [24]:
torch.manual_seed(42)
a: torch.Tensor = torch.tril(torch.ones(3, 3, device=device))
a /= a.sum(dim=1, keepdim=True)
b = torch.randint(0, 10, (3, 2), device=device).float()
c: torch.Tensor = a @ b
print(f"a=\n{a}")
print("------")
print(f"b=\n{b}")
print("------")
print(f"c=\n{c}")


a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]], device='cuda:0')
------
b=
tensor([[3., 9.],
        [8., 0.],
        [3., 7.]], device='cuda:0')
------
c=
tensor([[3.0000, 9.0000],
        [5.5000, 4.5000],
        [4.6667, 5.3333]], device='cuda:0')


In [25]:
# Version 2: Using a lower triangular matrix to calculate the mean of the previous time step.
# So the `x_bow: torch.Tensor = torch.zeros((B, T, C), device=device)` code above
# can be replaced with:
wei: torch.Tensor = torch.tril(torch.ones(T, T, device=device))
wei /= wei.sum(dim=1, keepdim=True)
x_bow2: torch.Tensor = wei @ x # (T, T) @ (B, T, C) = (B, T, T) @ (B, T, C) = (B, T, C)
torch.allclose(x_bow, x_bow2, atol=1e-7)  # check if they are the same

True

In [26]:
# Version 3: use softmax to calculate the mean of the previous time step.
tril: torch.Tensor = torch.tril(torch.ones(T, T, device=device))
wei: torch.Tensor = torch.zeros((T, T), device=device)
# fill all 0 (masked out) values with -inf
wei = wei.masked_fill((tril == 0), float("-inf"))
print(f"wei before softmax shape: {wei.shape}")
wei = wei.softmax(dim=-1)  # softmax of the row.
print(f"wei after softmax shape: {wei.shape}, x shape: {x.shape}")

x_bow3: torch.Tensor = wei @ x
x_bow_x_bow3_equal: bool = torch.allclose(x_bow, x_bow3)
x_bow2_x_bow3_equal: bool = torch.allclose(x_bow2, x_bow3)
print(f"x_bow and x_bow3 are equal: {x_bow_x_bow3_equal}, "
      f"max_diff: {torch.max(torch.abs(x_bow - x_bow3))}")
print(f"x_bow2 and x_bow3 are equal: {x_bow2_x_bow3_equal}, "
      f"max_diff: {torch.max(torch.abs(x_bow2 - x_bow3))}")


wei before softmax shape: torch.Size([8, 8])
wei after softmax shape: torch.Size([8, 8]), x shape: torch.Size([4, 8, 2])
x_bow and x_bow3 are equal: False, max_diff: 1.1920928955078125e-07
x_bow2 and x_bow3 are equal: True, max_diff: 0.0


### Single Head Self-attention

In [27]:
head_size: int = 16
# Convert token to a key matrix.
t_to_k: nn.Linear = nn.Linear(C, head_size, bias=False, device=x.device)
# Convert token to a query matrix.
t_to_q: nn.Linear = nn.Linear(C, head_size, bias=False, device=x.device)
# Convert token to a value matrix.
t_to_v: nn.Linear = nn.Linear(C, head_size, bias=False, device=x.device)

k: torch.Tensor = t_to_k(x)  # (B, T, C) -> (B, T, head_size)
q: torch.Tensor = t_to_q(x)  # (B, T, C) -> (B, T, head_size)
v: torch.Tensor = t_to_v(x)  # (B, T, C) -> (B, T, head_size)
print(f"k shape: {k.shape}, q shape: {q.shape}")
# (B, T, head_size) @ (B, head_size, T) = (B, T, T)
wei: torch.Tensor = q @ k.transpose(-2, -1)
print(f"wei shape: {wei.shape}")

tril: torch.Tensor = torch.tril(torch.ones(T, T, device=wei.device))
# This is the decoder part, for encoder, we don't need to mask the future tokens.
wei = wei.masked_fill(tril == 0, float("-inf"))
wei = wei.softmax(dim=-1)
print(f"wei shape after softmax: {wei.shape}")
out: torch.Tensor = wei @ v  # (B, T, T) @ (B, T, head_size) = (B, T, head_size)
print(f"out shape: {out.shape}")


k shape: torch.Size([4, 8, 16]), q shape: torch.Size([4, 8, 16])
wei shape: torch.Size([4, 8, 8])
wei shape after softmax: torch.Size([4, 8, 8])
out shape: torch.Size([4, 8, 16])


"Scaled" attention additional divides "wei" by 1 / sqrt(head_size). This makes it so when input Q, K are unit variance, the output is also unit variance. This is important for training stability.

In [28]:
k = torch.randn(B, T, head_size, device=device)
q = torch.randn(B, T, head_size, device=device)
wei = q @ k.transpose(-2, -1)
print(f"k var: {k.var()}")
print(f"q var: {q.var()}")
print(f"wei var: {wei.var()}")

k var: 1.0605725049972534
q var: 1.0459474325180054
wei var: 19.187782287597656


In [29]:
k = torch.randn(B, T, head_size, device=device)
q = torch.randn(B, T, head_size, device=device)
wei = q @ k.transpose(-2, -1) * head_size ** -0.5
print(f"k var: {k.var()}")
print(f"q var: {q.var()}")
print(f"wei var: {wei.var()}")

# Note the "14" vs "0.9b"

k var: 1.0497795343399048
q var: 1.1699278354644775
wei var: 1.2745065689086914


In [30]:
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1))
print(torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]) * 8, dim=-1))
# Note that the 2nd one has peak value.


tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [37]:
# Now investigate batch normalization.


class BatchNorm1d:

    def __init__(
        self, dim: int, eps: float = 1e-5, momentum: float = 0.1
    ) -> None:
        # Hyperparameters
        self.eps: float = eps

        # Parameters (trainable):
        # gamma and beta are like the weight and bias in a linear layer
        # gamma: scale parameter
        self.gamma = torch.ones(dim).to(device)
        # beta: shift parameter
        self.beta = torch.zeros(dim).to(device)

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        x_mean = x.mean(1, keepdim=True)
        x_var = x.var(1, keepdim=True, unbiased=True)
        x_norm = (x - x_mean) / torch.sqrt(x_var + self.eps)
        x_norm_scaled: Final[torch.Tensor] = self.gamma * x_norm + self.beta

        self.out: torch.Tensor = x_norm_scaled
        return x_norm_scaled

torch.manual_seed(1337)
module: BatchNorm1d = BatchNorm1d(100)
# batch size of 32, 100-dimensional vector
x: torch.Tensor = torch.randn(32, 100, device=device)
x = module(x)
x.shape

torch.Size([32, 100])

In [38]:
# x[:, 0].mean(), x[:, 0].std()
print(f"batch mean: {x[:, 0].mean()}")
print(f"batch std: {x[:, 0].std()}")


batch mean: -0.16268089413642883
batch std: 0.9686696529388428


In [39]:
# x[0, :].mean(), x[0, :].std()
print(f"layer mean: {x[0, :].mean()}")
print(f"layer std: {x[0, :].std()}")

layer mean: 1.3113021779531664e-08
layer std: 0.9999955296516418
